In [139]:
import plotly.express as px
import numpy as np
df = px.data.gapminder().query("year == 2007")
fig = px.treemap(df, path=[px.Constant("world"), 'continent', 'iso_alpha'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df['lifeExp'], weights=df['pop']))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [137]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32
...,...,...,...,...,...,...,...,...
1655,Vietnam,Asia,2007,74.249,85262356,2441.576404,VNM,704
1667,West Bank and Gaza,Asia,2007,73.422,4018332,3025.349798,PSE,275
1679,"Yemen, Rep.",Asia,2007,62.698,22211743,2280.769906,YEM,887
1691,Zambia,Africa,2007,42.384,11746035,1271.211593,ZMB,894


In [6]:
import plotly.express as px
fig = px.treemap(
    names = ["Eve","Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
    parents = ["", "Eve", "Eve", "Eve", "Seth", "Eve", "Eve", "Awan", "Eve"]
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [112]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/sales_success.csv')
print(df.head())

levels = ['salesperson', 'county', 'region'] # levels used for the hierarchical chart
color_columns = ['sales', 'calls']
value_column = 'calls'

def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]] / dfg[color_columns[1]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='total', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns[0]].sum() / df[color_columns[1]].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees


df_all_trees = build_hierarchical_dataframe(df, levels, value_column, color_columns)
average_score = df['sales'].sum() / df['calls'].sum()

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],)

fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',
    name=''
    ), 1, 1)

fig.add_trace(go.Treemap(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',
    maxdepth=2
    ), 1, 2)

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

   Unnamed: 0 region   county salesperson  calls  sales
0           0  North   Dallam          JE     35     23
1           1  North   Dallam          ZQ     49     13
2           2  North   Dallam          IJ     20      6
3           3  North  Hartley          WE     39     37
4           4  North  Hartley          PL     42     37


In [12]:
import plotly.graph_objects as go

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/96c0bd/sunburst-coffee-flavors-complete.csv')

fig = go.Figure()

fig.add_trace(go.Treemap(
    ids = df.ids,
    labels = df.labels,
    parents = df.parents,
    maxdepth=3,
    root_color="lightgrey"
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))

fig.show()


In [13]:
df.head()

,ids,labels,parents
0,Coffee,Coffee Flavors,NaN
1,Aromas,Aromas,Coffee
2,Tastes,Tastes,Coffee
3,Aromas-Enzymatic,Enzymatic,Aromas
4,Aromas-Sugar Browning,Sugar Browning,Aromas


### Tree map for real dataset 

In [144]:
import plotly.graph_objects as go 
from plotly.subplots import make_subplots 
import pandas as pd 

df = pd.read_pickle('explore_dataset/listings.pkl')

In [148]:
df.dropna(subset=['room_type','host_is_superhost', 'neighbourhood_group_cleansed'], inplace=True)
df = df.reset_index()

In [152]:
df['num_count'] = pd.Series(np.repeat(1, len(df)))
df.num_count
df = df[['room_type','host_is_superhost', 'neighbourhood_group_cleansed', 'num_count']]

In [158]:
levels = ['room_type', 'host_is_superhost', 'neighbourhood_group_cleansed']

def build_hierarchical_dataframe_modified(df, levels):
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg['num_count'] 
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(
        dict(
            id='total', 
            parent='', 
            value=len(df),
        )
    )
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees
df_all_trees = build_hierarchical_dataframe_modified(df, levels)
df_all_trees


,id,parent,value
0,Entire home/apt,f,536
1,Entire home/apt,f,6145
2,Entire home/apt,f,9115
3,Entire home/apt,f,1926
4,Entire home/apt,f,158
5,Entire home/apt,t,200
6,Entire home/apt,t,2009
7,Entire home/apt,t,1747
8,Entire home/apt,t,810
9,Entire home/apt,t,115


In [157]:
fig = go.Figure()

fig.add_trace(
    go.Treemap(
        labels=df_all_trees['id'],
        parents=df_all_trees['parent'],
        values=df_all_trees['value'],
        branchvalues='total',
        hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br>',
        maxdepth=-1,
        name='',
    )
)
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [59]:
df.groupby(levels[2:]).count().reset_index()

,neighbourhood_group_cleansed,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,price_cleansed
0,Bronx,1567,1567,1567,1567,1567,1565,1510,915,1567,...,1283,1283,1283,1567,1567,1567,1567,1567,1295,1567
1,Brooklyn,14841,14841,14841,14841,14841,14838,14567,9031,14841,...,11958,11952,11952,14841,14841,14841,14841,14841,12133,14841
2,Manhattan,16824,16824,16824,16824,16824,16816,16551,9386,16824,...,12492,12487,12487,16824,16824,16824,16824,16824,12718,16824
3,Queens,6174,6174,6174,6174,6174,6174,5976,3860,6174,...,4933,4932,4932,6174,6174,6174,6174,6174,4977,6174
4,Staten Island,446,446,446,446,446,446,432,274,446,...,379,379,379,446,446,446,446,446,380,446
